In [1]:
from numpy import *
import numpy as np
import healpy as hp
import time
import astropy
from astropy.time import Time
from astropy.coordinates import SkyCoord

In [2]:
dir='~/'

In [3]:
def rot_map(basemap,ra_rot):
    """Function takes in two arguments, an initial map and a rotation matrix"""
    nside=hp.get_nside(basemap) #get pixel size from basemap
    rotmap=zeros(hp.nside2npix(nside)) #creates empty map of nside pixels
    theta,phi=hp.pix2ang(nside,np.arange(hp.nside2npix(nside))) #transform pixel to angle. takes nside and the pixel indices of a given nside 
    phi=phi+ra_rot #rotates current value of right ascension (phi) by ra_rot
    rotmap[hp.ang2pix(nside,theta,phi)]=basemap #creates new rotated map with rotated values for right ascension 
    return rotmap

In [4]:
def vampire(time,nside,dead_zone):
    """This function blocks the sun in plot"""
    t = Time(time, format='mjd', scale='utc')
    sun=astropy.coordinates.get_sun(t) #get coordinates of sun at specific time
    theta_sun=pi/2.-sun.dec.radian #get declination in radians
    phi_sun=sun.ra.radian #get right ascension in radians
    v_sun=array([sin(theta_sun)*cos(phi_sun),sin(theta_sun)*sin(phi_sun),cos(theta_sun)])
    theta,phi=hp.pix2ang(nside,arange(hp.nside2npix(nside)))
    v_pix=array([sin(theta)*cos(phi),sin(theta)*sin(phi),cos(theta)])
    shield=dot(v_sun,v_pix)<=cos(dead_zone/180.*pi)
    return shield

In [ ]:
def bias_factor(hr,amplitude):
    """This function introduces daily cycle nonuniformitivity"""
    return  (1. + amplitude*sin(2.*pi*hr/24. - pi/8.))/24.

In [ ]:
basemap_cbc=hp.fitsfunc.read_map('obsbias/data/skyprior_cbc_hl.fits')
basemap_cbc=basemap_cbc/sum(basemap_cbc)